In [1]:
# import packages
import pandas as pd
import time
import datetime
import numpy as np
import matplotlib.pyplot as plt
import chromedriver_binary
import random
from datetime import timedelta, date
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import NoSuchElementException

In [2]:
#parametros e site
params = {'login':'04222526380',
          'password':'339715'}

driver = webdriver.Edge() # escolhendo o driver
driver.get("https://radar.serpro.gov.br/")# select site


In [6]:
def login():
  currentUrl = driver.current_url;
  if currentUrl == "https://radar.serpro.gov.br/main.html#/login":
    try:
      element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'usuario')))       
      element.send_keys(params['login'])
      element = driver.find_element(By.ID, "password")
      element.send_keys(params['password'])
      element = driver.find_element_by_id('Entrar')
      element.click()
      print("login feito")
      #acess the search of the taxs
      element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 
      'body > div.bg > div > div.bg.ng-scope > div.ng-scope > header > nav > div > ul > li.menu-principal.menu-dropdown')))
      element.click()
      element = WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.CSS_SELECTOR, 
      'body > div > div > div.bg.ng-scope > div.ng-scope > header > nav > div > ul > li.menu-principal.menu-dropdown.open > ul > div > li:nth-child(1) > ul > li:nth-child(1) > a')))
      element.click()
      element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.ID,'abaBuscaAvancada')))
      element.click()
      print("Acesso Menu realizado")
    except Exception as e:
      print("Não foi possível fazer login")
      print(e)
  print("Operação completa")
  
login() 

C:\Users\Filipe\AppData\Local\Temp\ipykernel_9340\3153827461.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id('Entrar')


login feito
Acesso Menu realizado
Operação completa


In [3]:
df_april = pd.read_excel("ResultadoConsultaInfracoes_abril.xlsx", skiprows=5)
df_april.columns = df_april.iloc[0]
df_april = df_april[1:]
df_test = df_april['N. Auto'].head(10)
# for i in df_test:
#     print(str(i))   

c:\Users\Filipe\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
sne_list = []
site_list = []
date_hour_list = []
date_list = []
hour_list = []
code_list = []
reg_list = []
jud_list = []

def reg_():
    reg = [reg.split(': ', 1)[1] for reg in reg_list]
    return(reg)

def site_():
    site = [site.split(': ', 1)[1] for site in site_list]
    return(site)

def code_():
    codes = [code.replace(' - ', '') for code in code_list]
    code = [code.split(': ', 1)[1] for code in codes]
    return(code)

def date_():
    date_hour = [date_hour.split(': ', 1)[1] for date_hour in date_hour_list]
    date = [date.split(' ', 1)[0] for date in date_hour]
    return(date)

def hour_():
    date_hour = [date_hour.split(': ', 1)[1] for date_hour in date_hour_list]
    hour = [hour.split(' ', 1)[1] for hour in date_hour]
    return(hour)

def souce_finn(auto):
    try:
        print("Inicio Pesquisa")
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'codigo')))
        element.clear()
        element.send_keys(str(auto))
        element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnBuscaAvancada')))
        time.sleep(0.75)
        element.click()
        element = WebDriverWait(driver, 12).until(EC.invisibility_of_element((By.CLASS_NAME, 'loading-root')))
        table_id = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#vm\.id > tbody')))

        print("Informação SEI:")
        rows = table_id.find_elements(By.TAG_NAME, "tr") 
        for row in rows:
            col_11 = row.find_elements(By.TAG_NAME, "td")[11]
            print(col_11.text)
            sne_list.append(col_11.text)

        print("Acessando a caixa de informações")
        element = WebDriverWait(driver, 12).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#vm\.id > tbody > tr > td:nth-child(2) > a')))
        time.sleep(0.75)
        element.click()
        element = WebDriverWait(driver, 12).until(EC.invisibility_of_element((By.CLASS_NAME, 'loading-root')))

        # get site
        site = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[1]')[0].text
        if site == '':
            site = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[2]')[0].text
        if site == '':
            site = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[2]')[0].text
        print(site)
        site_list.append(site) 
        
        # get code 
        code = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[1]')[0].text
        if code == '':
            code =  driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[7]/div/div[1]')[0].text
        print(code)
        code_list.append(code)
        
        # get matricula
        reg = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[7]')[0].text
        if "Matrícula" not in reg:
            reg = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[7]/div/div[7]')[0].text
        print(reg)
        reg_list.append(reg)

        # get  date and hours
        date_hour = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[5]')[0].text
        if date_hour == '':
            date_hour = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[5]')[0].text
        print(date_hour)
        date_hour_list.append(date_hour)
               
        driver.find_element_by_css_selector("#btnFecharDetalhe").click()
        
    except Exception as e:
      print("Acesso Pesquisa")
      print(e)
      print("Operação completa")

In [11]:

for i in df_test:
    print("Auto:", i)
    souce_finn(i)

reg_list = reg_()
site_list = site_()
code_list = code_()
date_list = date_()
hour_list = hour_()

Auto: ER00010639
Inicio Pesquisa
Informação SEI:
NÃO
Acessando a caixa de informações


C:\Users\Filipe\AppData\Local\Temp\ipykernel_9340\945133349.py:59: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  site = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[5]/div/div[1]')[0].text
C:\Users\Filipe\AppData\Local\Temp\ipykernel_9340\945133349.py:68: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  code = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[1]')[0].text
C:\Users\Filipe\AppData\Local\Temp\ipykernel_9340\945133349.py:75: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  reg = driver.find_elements_by_xpath('//*[@id="imprimirMulta"]/fieldset[6]/div/div[7]')[0].text
C:\Users\Filipe\AppData\Local\Temp\ipykernel_9340\945133349.py:82: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By

Local: Av. Maranhao, prox ao n. 2141
Código: 7463 - 0
Matrícula do Agente: 10750
Data/Hora: 30/04/2022 23:58:47
Auto: ER00010638
Inicio Pesquisa
Informação SEI:
NÃO
Acessando a caixa de informações
Local: AVENIDA JOSE FRANCISCO DE ALMEIDA NETO, N. 2767
Código: 7455 - 0
Matrícula do Agente: 10750
Data/Hora: 30/04/2022 23:50:05
Auto: ER00010637
Inicio Pesquisa
Informação SEI:
NÃO
Acessando a caixa de informações
Local: Av. Maranhao, prox ao n. 2141
Código: 7455 - 0
Matrícula do Agente: 10750
Data/Hora: 30/04/2022 23:36:54
Auto: ER00010636
Inicio Pesquisa
Informação SEI:
NÃO
Acessando a caixa de informações
Local: AVENIDA JOSE FRANCISCO DE ALMEIDA NETO, N. 2767
Código: 7463 - 0
Matrícula do Agente: 10750
Data/Hora: 30/04/2022 23:18:13
Auto: ER00010635
Inicio Pesquisa
Informação SEI:
SIM
Acessando a caixa de informações
Local: AV. PRESIDENTE KENNEDY, PROX AO COND TERRA DOS PASSAROS
Código: 7455 - 0
Matrícula do Agente: 10750
Data/Hora: 30/04/2022 23:11:59
Auto: ER00010634
Inicio Pesquisa
I